In [1]:
from dotenv import load_dotenv
import os
from web3 import Web3
from eth_account import Account
import pandas as pd
import time
import random
import math
from diskcache import Cache
import datetime as dt
from datetime import timedelta


load_dotenv()

True

In [2]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
flipside_api_key = os.getenv("FLIPSIDE_API_KEY")

ACCOUNT_ADDRESS = os.getenv('ACCOUNT_ADDRESS')
PRIVATE_KEY = os.getenv('PRIVATE_KEY')
GATEWAY = os.getenv('ARBITRUM_GATEWAY')

In [3]:
from openai import OpenAI
openai_client = OpenAI(api_key=OPENAI_API_KEY)

In [4]:
os.chdir('..')

In [5]:
from python_scripts.web3_utils import *
from python_scripts.apis import token_classifier_portfolio, flipside_api_results
from python_scripts.macro_data import main as macro_main
from sql_queries.queries import latest_portfolio_metrics

2025-01-11 12:19:30.268 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


Current Directory: E:\Projects\encode hackathon


In [6]:
cache = Cache('classifier_data')

In [7]:
def network(chain):
        if chain == 'gnosis':
            primary_gateway = GNOSIS_GATEWAY  # Replace with your Infura URL
            backup_gateway = 'https://lb.nodies.app/v1/406d8dcc043f4cb3959ed7d6673d311a'  # Your backup gateway
        elif chain == 'arbitrum':
            primary_gateway = GATEWAY  # Replace with your Infura URL
            backup_gateway = GATEWAY
        elif chain == 'optimism':
            primary_gateway = OPTIMISM_GATEWAY  # Replace with your Infura URL
            backup_gateway = OPTIMISM_GATEWAY
        elif chain == 'ethereum':
            primary_gateway = ETHEREUM_GATEWAY  # Replace with your Infura URL
            backup_gateway = ETHEREUM_GATEWAY

        print(f'Gateway: {primary_gateway}')

        for gateway in [primary_gateway, backup_gateway]:
            w3 = Web3(Web3.HTTPProvider(gateway))
            if w3.is_connected():
                try:
                    latest_block = w3.eth.get_block('latest')['number']  # Only try this if connected
                    print(f"Connected to {chain} via {gateway}: {latest_block} block")
                    return w3, gateway
                except Exception as e:
                    print(f"Connected to {gateway} but failed to fetch latest block. Error: {e}")
            else:
                print(f"Failed to connect to {chain} via {gateway}. Trying next gateway...")

        raise ConnectionError(f"Failed to connect to {chain} network using both primary and backup gateways.")

In [8]:
chain = 'arbitrum'

w3, gateway = network(chain)

account = Account.from_key(PRIVATE_KEY)
w3.eth.default_account = account.address

Gateway: https://arb-mainnet.g.alchemy.com/v2/sDx1aRO1I82_YWxxUK9S_jnDXHTXEqFl
Connected to arbitrum via https://arb-mainnet.g.alchemy.com/v2/sDx1aRO1I82_YWxxUK9S_jnDXHTXEqFl: 294373183 block


In [9]:
# import json

# # Define the path to your ABI file
# mlAMPL_abi_file_path = "abi/mlAMPL.json"
# WETH_abi_file_path = "abi/weth_abi.json"
# bonding_abi_file_path = "abi/bonding_contract_abi.json"
# lp_manager_path = 'abi/nftmanager.json'
# staking_path = 'abi/staking_abi.json'
# erc20_abi_path = 'abi/erc20_abi.json'
# nft_factory_path = 'abi/nft_factory.json'
# pool_abi_path = 'abi/pool_abi.json'

# abi_paths = [mlAMPL_abi_file_path, WETH_abi_file_path, bonding_abi_file_path,lp_manager_path,staking_path,erc20_abi_path,nft_factory_path,pool_abi_path]
# abis = {}

# for path in abi_paths:
#     with open(path, "r") as file:
#         abis[path] = json.load(file)

# print(abis)


In [10]:
def data_cleaning(df,dropna=True,ffill=False):
    clean_df = clean_prices(df)
    clean_df = to_time(clean_df)
    if dropna == True:
        # clean_df = clean_df.dropna(axis=1, how='any')
        clean_df = clean_df.dropna()
    if ffill == True:
        clean_df = clean_df.resample('h').ffill().bfill()

    if '__row_index' in clean_df.columns:
        clean_df.drop(columns=['__row_index'], inplace=True)

    return clean_df

def to_time(df):
    time_cols = ['date','dt','hour','time','day','month','year','week','timestamp','date(utc)','block_timestamp']
    for col in df.columns:
        if col.lower() in time_cols and col.lower() != 'timestamp':
            df[col] = pd.to_datetime(df[col])
            df.set_index(col, inplace=True)
        elif col.lower() == 'timestamp':
            df[col] = pd.to_datetime(df[col], unit='ms')
            df.set_index(col, inplace=True)
    print(df.index)
    return df 

def clean_prices(prices_df):
    print('cleaning prices')
    # Pivot the dataframe
    prices_df = prices_df.drop_duplicates(subset=['hour', 'symbol'])
    prices_df_pivot = prices_df.pivot(
        index='hour',
        columns='symbol',
        values='price'
    )
    prices_df_pivot = prices_df_pivot.reset_index()

    # Rename the columns by combining 'symbol' with a suffix
    prices_df_pivot.columns = ['hour'] + [f'{col}_Price' for col in prices_df_pivot.columns[1:]]
    
    print(f'cleaned prices: {prices_df_pivot}')
    return prices_df_pivot

In [11]:
def pull_data(function,path,model_name, api=False,start_date=None):
    print(f'function:{function},start_date:{start_date},path:{path},api:{api},model_name: {model_name}')

    if api:
        print(f'api True')
        # Parse dates into datetime format for consistency
        start_date = dt.datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
        
        # Use formatted date strings as needed in the dao_advisor_portfolio and lst_portfolio_prices functions
        prices = function(start_date.strftime('%Y-%m-%d %H:%M:%S'))
        
        prices_df = flipside_api_results(prices, flipside_api_key)

        prices_df.to_csv(path)
    else:
        print(f'api False')
        prices_df = pd.read_csv(path)

    dataset = {
        f'portfolio': prices_df
    }

    return dataset

In [12]:
def prices_data_func(network,
                     api_key,use_cached_data,name,days=None,
                     function=None,start_date=None,
                     backtest_period=None,filtered_assets=None):
    
    if start_date is None and backtest_period is None:
        raise KeyError("Provide either a start date or backtest_period")
    
    print(f"backtest days: {(pd.to_datetime(dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d %H:00:00')) - pd.to_datetime(start_date)).days}")
    
    if backtest_period is None:
        backtest_period = (pd.to_datetime(dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d %H:00:00')) - pd.to_datetime(start_date)).days * 24
        if backtest_period < 1:
            backtest_period = 1

    if function is None:

        data = token_classifier_portfolio(
            network=network,
            days=days,
            name=name,
            api_key = api_key,
            use_cached_data=use_cached_data,
            start_date = start_date,
            prices_only=True
        )

        prices_df = data_cleaning(data['portfolio'])
        prices_df
    else: 
        data = pull_data(function=function,start_date=start_date, path=f'data/{name}.csv', api=not use_cached_data,model_name=name)
        prices_df = data_cleaning(data['portfolio'])
        prices_df = prices_df[prices_df.index >= start_date].dropna()
        prices_df

    # prices_df.columns = prices_df.columns.str.replace('_Price','')
    filtered_assets_with_price = [f"{asset}_Price" for asset in filtered_assets]


    return data, prices_df[filtered_assets_with_price]

In [13]:
def prepare_data_for_simulation(price_timeseries, start_date, end_date):
    """
    Ensure price_timeseries has entries for start_date and end_date.
    If not, fill in these dates using the last available data.
    """
    # Ensure 'ds' is in datetime format
    # price_timeseries['hour'] = pd.to_datetime(price_timeseries['hour'])
    
    # Set the index to 'ds' for easier manipulation
    # if price_timeseries.index.name != 'hour':
    #     price_timeseries.set_index('hour', inplace=True)

    print(f'price index: {price_timeseries.index}')

    price_timeseries.index = price_timeseries.index.tz_localize(None)
    
    # Check if start_date and end_date exist in the data
    required_dates = pd.date_range(start=start_date, end=end_date, freq='H')
    all_dates = price_timeseries.index.union(required_dates)
    
    # Reindex the dataframe to ensure all dates from start to end are present
    price_timeseries = price_timeseries.reindex(all_dates)
    
    # Forward fill to handle NaN values if any dates were missing
    price_timeseries.fillna(method='ffill', inplace=True)

    # Reset index if necessary or keep the datetime index based on further needs
    # price_timeseries.reset_index(inplace=True, drop=False)
    # price_timeseries.rename(columns={'index': 'hour'}, inplace=True)
    # price_timeseries.set_index('hour',inplace=True)
    
    return price_timeseries


In [14]:
network = chain

model = f'{network}_classifier'

params = cache.get(f'{model} params')
classifier_data = cache.get(f'{model}_portfolio')
original_prices_df = cache.get(f'{model}_prices')
days = params['days']

In [15]:
params

{'model': 'arbitrum_classifier',
 'days': 7,
 'top': 20,
 'network': 'arbitrum',
 'backtest_period': 4380,
 'start_date': '2024-07-08'}

In [16]:
classifier_data

,symbol,token_address,sharpe_ratio,excess_return,latest_price,latest_hour,stddev_30d,volume,average_order,sixty_d_return,__row_index,original_latest_hour
0,LAVA,0x11e969e9b3f89cb16d686a03cd8508c9fc0361af,31.123394,0.891667,0.173335,2025-01-06 16:00:00+00:00,0.028649,2.867698e+06,163.262081,0.892469,0,2025-01-06 16:00:00+00:00
1,MAGIC,0x539bde0d7dbd336b79148aa742883198bbf60342,4.465459,0.399587,0.611536,2025-01-06 16:00:00+00:00,0.089484,8.264843e+06,271.361034,0.400389,1,2025-01-06 16:00:00+00:00
2,STG,0x6694340fc020c5e6b96567843da2df01b2ce1eb6,4.262890,0.228900,0.491961,2025-01-06 16:00:00+00:00,0.053696,2.324067e+06,192.341905,0.229703,2,2025-01-06 16:00:00+00:00
3,USDC,0xaf88d065e77c8cc2239327c5edb3a432268e5831,3.525502,0.003190,1.006000,2025-01-06 16:00:00+00:00,0.000905,8.806350e+08,1827.297492,0.003992,3,2025-01-06 16:00:00+00:00
4,DAI,0xda10009cbd5d07dd0cecc66161fc93d7c9000da1,3.177627,0.005553,1.005000,2025-01-06 16:00:00+00:00,0.001748,1.192044e+07,1055.560284,0.006356,4,2025-01-06 16:00:00+00:00
5,USDC,0xff970a61a04b1ca14834a43f5de4533ebddb5cc8,2.392183,0.004193,1.006000,2025-01-06 16:00:00+00:00,0.001753,1.330199e+08,831.166548,0.004995,5,2025-01-06 16:00:00+00:00
6,ARB,0x912ce59144191c1204e64559fe8253a0e49e6548,2.121842,0.271495,0.943365,2025-01-06 16:00:00+00:00,0.127952,1.828342e+08,820.870725,0.272297,6,2025-01-06 16:00:00+00:00
7,USDT,0xfd086bc7cd5c481dcc9c85ebe478a1c0b69fcbb9,0.905344,0.001566,1.002000,2025-01-06 16:00:00+00:00,0.001729,4.616128e+08,1353.754784,0.002368,7,2025-01-06 16:00:00+00:00
8,CRV,0x11cdb42b0eb46d95f990bedd4695a6e3fa034978,0.605335,0.069327,1.011000,2025-01-06 16:00:00+00:00,0.114526,9.694287e+06,927.682998,0.070129,8,2025-01-06 16:00:00+00:00
9,ZRO,0x6985884c4392d348587b19cb9eaaf157f13271cd,0.244965,0.136865,5.950000,2025-01-06 16:00:00+00:00,0.558713,6.521666e+06,468.611463,0.137667,9,2025-01-06 16:00:00+00:00


In [17]:
full_start = original_prices_df.index.min().strftime('%Y-%m-%d %H:00:00')
full_start

'2024-07-30 13:00:00'

In [18]:
filtered_assets = classifier_data['symbol'].unique()
filtered_assets

array(['LAVA', 'MAGIC', 'STG', 'USDC', 'DAI', 'ARB', 'USDT', 'CRV', 'ZRO'],
      dtype=object)

In [19]:
data_start_date = dt.datetime.now(dt.timezone.utc) - timedelta(hours=5)
data_start_date = data_start_date.strftime('%Y-%m-%d %H:00:00')

today_utc = dt.datetime.now(dt.timezone.utc) 
formatted_today_utc = today_utc.strftime('%Y-%m-%d %H:00:00')

data_version = dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d %H-00-00')
data_version_comp = dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d %H:00:00') 

In [20]:
macro_data = macro_main(formatted_today_utc)

Fetching Interest Rate Data...


OpenBBError: 
[Unexpected Error] -> Exception -> Error with the OECD request: 429

In [21]:
start_date = str(data_start_date)
end_date = dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d %H:00:00') 

start_date

'2025-01-11 12:00:00'

In [22]:
end_date

'2025-01-11 17:00:00'

In [23]:
data, prices_df = prices_data_func(
                            network=network, 
                            name=model,
                            api_key=flipside_api_key,
                            use_cached_data=False,
                            function=None,
                            start_date=start_date,
                            filtered_assets=filtered_assets
                            )
        
prices_df = prepare_data_for_simulation(prices_df, start_date, end_date)

backtest days: 0
use_cached_data: False
volume_threshold: 1
start_date: 2025-01-11 12:00:00
data_start_str: 2024-07-08 04:00:00
Beginning: '2025-01-11 12:00:00'
Query not completed. Retrying in 30 seconds...
cleaning prices
cleaned prices:                        hour  ARB_Price  CRV_Price  DAI_Price  LAVA_Price  \
0  2025-01-11T12:00:00.000Z   0.736588   0.819947   1.000000    0.143798   
1  2025-01-11T13:00:00.000Z   0.740618   0.825797   0.998560    0.146932   
2  2025-01-11T14:00:00.000Z   0.737405   0.819361   0.999276    0.146359   
3  2025-01-11T15:00:00.000Z   0.732249   0.811187   1.000000    0.146579   
4  2025-01-11T16:00:00.000Z   0.732417   0.814135   1.000000    0.153532   

   MAGIC_Price  STG_Price  USDC_Price  USDT_Price  ZRO_Price  
0     0.476683   0.407951    1.000000    1.000000       4.66  
1     0.480520   0.409601    1.000000    0.999616       4.69  
2     0.474871   0.409888    0.999998    0.999796       4.68  
3     0.471421   0.410636    0.999332    0.999450  

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_55368\3188632879.py:18: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  required_dates = pd.date_range(start=start_date, end=end_date, freq='H')
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_55368\3188632879.py:25: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  price_timeseries.fillna(method='ffill', inplace=True)


In [24]:
# prices_df.columns

In [25]:
# import pandas as pd
# print(pd.__version__)


In [26]:
prices_df.columns = [col.replace('_Price', '') for col in prices_df.columns]
prices_df

,LAVA,MAGIC,STG,USDC,DAI,ARB,USDT,CRV,ZRO
2025-01-11 12:00:00,0.143798,0.476683,0.407951,1.000000,1.000000,0.736588,1.000000,0.819947,4.66
2025-01-11 13:00:00,0.146932,0.480520,0.409601,1.000000,0.998560,0.740618,0.999616,0.825797,4.69
2025-01-11 14:00:00,0.146359,0.474871,0.409888,0.999998,0.999276,0.737405,0.999796,0.819361,4.68
2025-01-11 15:00:00,0.146579,0.471421,0.410636,0.999332,1.000000,0.732249,0.999450,0.811187,4.64
2025-01-11 16:00:00,0.153532,0.472452,0.413655,0.999745,1.000000,0.732417,0.999651,0.814135,4.63
2025-01-11 17:00:00,0.153532,0.472452,0.413655,0.999745,1.000000,0.732417,0.999651,0.814135,4.63


In [27]:
w3

In [28]:
def rebalance_portfolio(
    uniswap, 
    token_contracts, 
    token_decimals, 
    target_compositions, 
    account_address, 
):
    """
    Rebalances the portfolio by selling all tokens into WETH and then buying target allocations using WETH.

    Parameters:
    - uniswap: Initialized Uniswap class instance.
    - token_contracts: Dict of token addresses.
    - token_decimals: Dict of token decimals.
    - target_compositions: Dict of target compositions as fractions summing to 1.
    - account_address: ETH wallet address.
    - web3: Initialized Web3 instance.
    """

    # WETH address and checksum
    WETH_ADDRESS = '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'
    checksum_weth_address = Web3.to_checksum_address(WETH_ADDRESS)

    # Step 1: Convert Token Addresses to Checksum Format
    checksum_addresses = {token: Web3.to_checksum_address(address) for token, address in token_contracts.items()}

    # Step 2: Sell All Current Token Holdings into WETH
    for token, address in checksum_addresses.items():
        try:
            balance_wei = uniswap.get_token_balance(address)
            balance = balance_wei / 10**token_decimals[token]
            
            # Adjust the balance to avoid precision issues (round down to 6 decimal places)
            adjusted_balance = math.floor(balance * 10**8) / 10**8
            
            if adjusted_balance > 0:
                amount_to_sell = int(adjusted_balance * 10**token_decimals[token])
                print(f"Selling {adjusted_balance:.6f} {token} for WETH")
                uniswap.make_trade(
                    checksum_addresses[token],
                    checksum_weth_address,  # WETH as output token
                    amount_to_sell
                )
                wait_time = random.randint(15, 30)
                print(f"Waiting {wait_time} seconds before the next call...")
                time.sleep(wait_time)
        except Exception as e:
            print(f"Error selling {token}: {e}")

    # Step 3: Get Current WETH Balance
    weth_balance_wei = uniswap.get_token_balance(checksum_weth_address)
    weth_balance = weth_balance_wei / 10**18
    print(f"Total WETH balance after selling: {weth_balance:.6f} WETH")

    # Step 4: Buy Target Tokens Based on Target Compositions
    for token, target_weight in target_compositions.items():
        if target_weight > 0:
            weth_to_spend = weth_balance * target_weight
            
            # Adjust the WETH amount to avoid precision issues (round down to 6 decimal places)
            adjusted_weth_to_spend = math.floor(weth_to_spend * 10**8) / 10**8

            if adjusted_weth_to_spend <= 0:
                continue

            try:
                print(f"Buying {token} with {adjusted_weth_to_spend:.6f} WETH")

                uniswap.make_trade(
                    checksum_weth_address,        # WETH as input token
                    checksum_addresses[token],    # Target token
                    int(adjusted_weth_to_spend * 10**18),  # Convert WETH amount to wei
                    fee=3000                      # Assuming 0.3% fee pool for Uniswap V3
                )

                wait_time = random.randint(15, 30)
                print(f"Waiting {wait_time} seconds before the next call...")
                time.sleep(wait_time)

            except Exception as e:
                print(f"Error buying {token}: {e}")

    # Step 5: Log the Rebalancing Info
    final_weth_balance = uniswap.get_token_balance(checksum_weth_address) / 10**18
    print(f"Final WETH balance: {final_weth_balance:.6f} WETH")

    rebal_info = {
        "account_address": account_address,
        "initial_weth_balance": weth_balance,
        "final_weth_balance": final_weth_balance,
        "purchases": target_compositions,
    }

    # Save rebalancing info to CSV
    rebal_df = pd.DataFrame([rebal_info])
    rebal_df.to_csv('data/live_rebal_results.csv', index=False)
    print("Rebalancing info saved to 'data/live_rebal_results.csv'.")

In [29]:
metrics_query = latest_portfolio_metrics(classifier_data['token_address'], network, days, dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d'))

In [30]:
latest_metrics_df = flipside_api_results(metrics_query, flipside_api_key)

Query not completed. Retrying in 30 seconds...


In [31]:
latest_metrics_df

,symbol,token_address,sharpe_ratio,excess_return,latest_price,latest_hour,stddev_30d,rolling_7d_avg,token_return,rolling_30d_avg,avg_vol,sum_vol,__row_index
0,USDT,0xfd086bc7cd5c481dcc9c85ebe478a1c0b69fcbb9,None,None,0.999651,2025-01-11T16:00:00.000Z,0.001438,0.999968,0.000317,0.999954,1409.524415,5.982867e+08,0
1,CRV,0x11cdb42b0eb46d95f990bedd4695a6e3fa034978,None,None,0.814135,2025-01-11T16:00:00.000Z,0.113305,0.920468,-0.253087,0.933541,888.868335,6.237189e+06,1
2,USDC,0xaf88d065e77c8cc2239327c5edb3a432268e5831,None,None,0.999916,2025-01-11T16:00:00.000Z,0.000846,1.000024,0.000457,1.000011,2024.853506,1.232919e+09,2
3,DAI,0xda10009cbd5d07dd0cecc66161fc93d7c9000da1,None,None,1.000000,2025-01-11T16:00:00.000Z,0.001610,1.000079,0.000019,1.000007,831.008586,1.122277e+07,3
4,LAVA,0x11e969e9b3f89cb16d686a03cd8508c9fc0361af,None,None,0.153532,2025-01-11T16:00:00.000Z,0.036791,0.165948,-0.010908,0.166013,216.160874,3.744555e+06,4
5,USDC,0xff970a61a04b1ca14834a43f5de4533ebddb5cc8,None,None,0.999745,2025-01-11T16:00:00.000Z,0.001697,1.000037,0.000879,1.000032,799.668146,1.584902e+08,5
6,ZRO,0x6985884c4392d348587b19cb9eaaf157f13271cd,None,None,4.630000,2025-01-11T16:00:00.000Z,0.602420,5.225089,-0.198962,5.267892,487.216524,6.117978e+06,6
7,MAGIC,0x539bde0d7dbd336b79148aa742883198bbf60342,None,None,0.472452,2025-01-11T16:00:00.000Z,0.071942,0.536947,-0.120031,0.538447,288.088149,5.764932e+06,7
8,STG,0x6694340fc020c5e6b96567843da2df01b2ce1eb6,None,None,0.413655,2025-01-11T16:00:00.000Z,0.058103,0.452631,-0.017479,0.453425,191.097720,2.173737e+06,8
9,ARB,0x912ce59144191c1204e64559fe8253a0e49e6548,None,None,0.732417,2025-01-11T16:00:00.000Z,0.099533,0.831742,-0.121736,0.832248,827.088733,2.209468e+08,9


In [32]:
portfolio = classifier_data[['symbol','token_address']]

TOKEN_CONTRACTS = {
    row['symbol']: row['token_address'] for _, row in portfolio.iterrows()
}

TOKEN_DECIMALS = get_token_decimals(TOKEN_CONTRACTS,w3)


TOKEN_CONTRACTS

{'LAVA': '0x11e969e9b3f89cb16d686a03cd8508c9fc0361af',
 'MAGIC': '0x539bde0d7dbd336b79148aa742883198bbf60342',
 'STG': '0x6694340fc020c5e6b96567843da2df01b2ce1eb6',
 'USDC': '0xff970a61a04b1ca14834a43f5de4533ebddb5cc8',
 'DAI': '0xda10009cbd5d07dd0cecc66161fc93d7c9000da1',
 'ARB': '0x912ce59144191c1204e64559fe8253a0e49e6548',
 'USDT': '0xfd086bc7cd5c481dcc9c85ebe478a1c0b69fcbb9',
 'CRV': '0x11cdb42b0eb46d95f990bedd4695a6e3fa034978',
 'ZRO': '0x6985884c4392d348587b19cb9eaaf157f13271cd'}

In [33]:
TOKEN_DECIMALS

{'LAVA': 6,
 'MAGIC': 18,
 'STG': 18,
 'USDC': 6,
 'DAI': 18,
 'ARB': 18,
 'USDT': 6,
 'CRV': 18,
 'ZRO': 18}

In [34]:
model_balances = get_balance(TOKEN_CONTRACTS,TOKEN_DECIMALS,ACCOUNT_ADDRESS,w3)

model_balances  

Balances for account 0xFA475d5FB90C4b90D929a64732DA6De38a966416: {'LAVA': 10.548743, 'MAGIC': 0.0, 'STG': 0.0, 'USDC': 0.0, 'DAI': 0.0, 'ARB': 0.0, 'USDT': 0.0, 'CRV': 0.0, 'ZRO': 0.0}


{'LAVA': 10.548743,
 'MAGIC': 0.0,
 'STG': 0.0,
 'USDC': 0.0,
 'DAI': 0.0,
 'ARB': 0.0,
 'USDT': 0.0,
 'CRV': 0.0,
 'ZRO': 0.0}

In [35]:
prices_df

,LAVA,MAGIC,STG,USDC,DAI,ARB,USDT,CRV,ZRO
2025-01-11 12:00:00,0.143798,0.476683,0.407951,1.000000,1.000000,0.736588,1.000000,0.819947,4.66
2025-01-11 13:00:00,0.146932,0.480520,0.409601,1.000000,0.998560,0.740618,0.999616,0.825797,4.69
2025-01-11 14:00:00,0.146359,0.474871,0.409888,0.999998,0.999276,0.737405,0.999796,0.819361,4.68
2025-01-11 15:00:00,0.146579,0.471421,0.410636,0.999332,1.000000,0.732249,0.999450,0.811187,4.64
2025-01-11 16:00:00,0.153532,0.472452,0.413655,0.999745,1.000000,0.732417,0.999651,0.814135,4.63
2025-01-11 17:00:00,0.153532,0.472452,0.413655,0.999745,1.000000,0.732417,0.999651,0.814135,4.63


In [36]:
TOKEN_CONTRACTS

{'LAVA': '0x11e969e9b3f89cb16d686a03cd8508c9fc0361af',
 'MAGIC': '0x539bde0d7dbd336b79148aa742883198bbf60342',
 'STG': '0x6694340fc020c5e6b96567843da2df01b2ce1eb6',
 'USDC': '0xff970a61a04b1ca14834a43f5de4533ebddb5cc8',
 'DAI': '0xda10009cbd5d07dd0cecc66161fc93d7c9000da1',
 'ARB': '0x912ce59144191c1204e64559fe8253a0e49e6548',
 'USDT': '0xfd086bc7cd5c481dcc9c85ebe478a1c0b69fcbb9',
 'CRV': '0x11cdb42b0eb46d95f990bedd4695a6e3fa034978',
 'ZRO': '0x6985884c4392d348587b19cb9eaaf157f13271cd'}

In [37]:
latest_prices = {
    token: float(prices_df[f"{token}"].iloc[-1])
    for token in TOKEN_CONTRACTS.keys()
    if f"{token}" in prices_df.columns
}

latest_prices

{'LAVA': 0.1535317779,
 'MAGIC': 0.472452,
 'STG': 0.413655,
 'USDC': 0.999745,
 'DAI': 1.0,
 'ARB': 0.7324165994,
 'USDT': 0.999651,
 'CRV': 0.814135,
 'ZRO': 4.63}

In [38]:
model_balances_usd = convert_to_usd(model_balances,latest_prices,TOKEN_CONTRACTS)
model_balances_usd

balances: dict_keys(['LAVA', 'MAGIC', 'STG', 'USDC', 'DAI', 'ARB', 'USDT', 'CRV', 'ZRO'])
TOKEN_CONTRACTS.keys(): dict_keys(['LAVA', 'MAGIC', 'STG', 'USDC', 'DAI', 'ARB', 'USDT', 'CRV', 'ZRO'])


{'LAVA': 1.6195672674001798,
 'MAGIC': 0.0,
 'STG': 0.0,
 'USDC': 0.0,
 'DAI': 0.0,
 'ARB': 0.0,
 'USDT': 0.0,
 'CRV': 0.0,
 'ZRO': 0.0}

In [39]:
available_balance = sum(model_balances_usd.values())
available_balance

1.6195672674001798

In [40]:
available_balance

1.6195672674001798

In [41]:
comp_dict = {
    f"{token}": balance_usd / available_balance
    for token, balance_usd in model_balances_usd.items()
}

# comp_dict["date"] = formatted_today_utc

In [42]:
comp_dict

{'LAVA': 1.0,
 'MAGIC': 0.0,
 'STG': 0.0,
 'USDC': 0.0,
 'DAI': 0.0,
 'ARB': 0.0,
 'USDT': 0.0,
 'CRV': 0.0,
 'ZRO': 0.0}

In [43]:
# update_historical_data(comp_dict)

portfolio_dict = {
    "Portfolio Value": available_balance,
    "date": formatted_today_utc
}

# update_portfolio_data(portfolio_dict)

In [44]:
identity_map = {
    "risk_level": "Moderate risk, willing to risk some money for the right investments but not chasing every new opportunity.",
    "token_preferences": "Only tokens appearing in the classifier portfolio.",
    "mission_statement": "Accumulate as much WETH as possible given the available funds.",
}

In [45]:
prices_df

,LAVA,MAGIC,STG,USDC,DAI,ARB,USDT,CRV,ZRO
2025-01-11 12:00:00,0.143798,0.476683,0.407951,1.000000,1.000000,0.736588,1.000000,0.819947,4.66
2025-01-11 13:00:00,0.146932,0.480520,0.409601,1.000000,0.998560,0.740618,0.999616,0.825797,4.69
2025-01-11 14:00:00,0.146359,0.474871,0.409888,0.999998,0.999276,0.737405,0.999796,0.819361,4.68
2025-01-11 15:00:00,0.146579,0.471421,0.410636,0.999332,1.000000,0.732249,0.999450,0.811187,4.64
2025-01-11 16:00:00,0.153532,0.472452,0.413655,0.999745,1.000000,0.732417,0.999651,0.814135,4.63
2025-01-11 17:00:00,0.153532,0.472452,0.413655,0.999745,1.000000,0.732417,0.999651,0.814135,4.63


In [46]:
classifier_data['latest_hour']

0    2025-01-06 16:00:00+00:00
1    2025-01-06 16:00:00+00:00
2    2025-01-06 16:00:00+00:00
3    2025-01-06 16:00:00+00:00
4    2025-01-06 16:00:00+00:00
5    2025-01-06 16:00:00+00:00
6    2025-01-06 16:00:00+00:00
7    2025-01-06 16:00:00+00:00
8    2025-01-06 16:00:00+00:00
9    2025-01-06 16:00:00+00:00
Name: latest_hour, dtype: object

In [47]:
params['days']

7

In [48]:
data_summary_list = []
for symbol in latest_metrics_df['symbol'].unique():
    row = latest_metrics_df[latest_metrics_df['symbol'] == symbol].iloc[0]
    text = (
        f"{symbol} stats as of {pd.to_datetime(row['latest_hour']).strftime('%Y-%m-%d %H:%M')}: "
        f"Sharpe Ratio: {row['sharpe_ratio']}, "
        f"Excess Return: {row['excess_return']}, "
        f"Latest Price: ${row['latest_price']}, "
        f"60D Return: {row['token_return']*100}%"
    )
    data_summary_list.append(text)

# Join the list into a single string with newlines
formatted_data_summary = "\n".join(data_summary_list)

In [49]:
formatted_data_summary

'USDT stats as of 2025-01-11 16:00: Sharpe Ratio: None, Excess Return: None, Latest Price: $0.999651, 60D Return: 0.031721126270000004%\nCRV stats as of 2025-01-11 16:00: Sharpe Ratio: None, Excess Return: None, Latest Price: $0.814135, 60D Return: -25.3087156%\nUSDC stats as of 2025-01-11 16:00: Sharpe Ratio: None, Excess Return: None, Latest Price: $0.999916, 60D Return: 0.04572473708%\nDAI stats as of 2025-01-11 16:00: Sharpe Ratio: None, Excess Return: None, Latest Price: $1.0, 60D Return: 0.001900036101%\nLAVA stats as of 2025-01-11 16:00: Sharpe Ratio: None, Excess Return: None, Latest Price: $0.1535317779, 60D Return: -1.090817941%\nZRO stats as of 2025-01-11 16:00: Sharpe Ratio: None, Excess Return: None, Latest Price: $4.63, 60D Return: -19.89619377%\nMAGIC stats as of 2025-01-11 16:00: Sharpe Ratio: None, Excess Return: None, Latest Price: $0.472452, 60D Return: -12.0030695%\nSTG stats as of 2025-01-11 16:00: Sharpe Ratio: None, Excess Return: None, Latest Price: $0.413655, 6

In [50]:
formatted_data_summary

'USDT stats as of 2025-01-11 16:00: Sharpe Ratio: None, Excess Return: None, Latest Price: $0.999651, 60D Return: 0.031721126270000004%\nCRV stats as of 2025-01-11 16:00: Sharpe Ratio: None, Excess Return: None, Latest Price: $0.814135, 60D Return: -25.3087156%\nUSDC stats as of 2025-01-11 16:00: Sharpe Ratio: None, Excess Return: None, Latest Price: $0.999916, 60D Return: 0.04572473708%\nDAI stats as of 2025-01-11 16:00: Sharpe Ratio: None, Excess Return: None, Latest Price: $1.0, 60D Return: 0.001900036101%\nLAVA stats as of 2025-01-11 16:00: Sharpe Ratio: None, Excess Return: None, Latest Price: $0.1535317779, 60D Return: -1.090817941%\nZRO stats as of 2025-01-11 16:00: Sharpe Ratio: None, Excess Return: None, Latest Price: $4.63, 60D Return: -19.89619377%\nMAGIC stats as of 2025-01-11 16:00: Sharpe Ratio: None, Excess Return: None, Latest Price: $0.472452, 60D Return: -12.0030695%\nSTG stats as of 2025-01-11 16:00: Sharpe Ratio: None, Excess Return: None, Latest Price: $0.413655, 6

In [51]:
assets = classifier_data['symbol'].unique()
assets

array(['LAVA', 'MAGIC', 'STG', 'USDC', 'DAI', 'ARB', 'USDT', 'CRV', 'ZRO'],
      dtype=object)

In [52]:
recent_headlines = ("*ROBINHOOD: CRYPTO TRADING VOLUMES OVER $30B, UP 600% YOY\n"
                     "*FED’S MUSALEM: TIME MAY BE APPROACHING TO SLOW OR PAUSE RATE CUTS\n")

chat_summary = "Bob is happy with the Ethereum roadmap and has been hearing more people talk about it.\n"

chat_and_data_summary = chat_summary + recent_headlines

In [53]:
user_message = (
    "# Instructions:\n"
    "Here are some details about my trading portfolio. Please help me make decisions to rebalance it based on the provided data.\n"
    "# Personality\n"
    f"{identity_map.get('chat_personality')}\n"
    "# Risk Level\n"
    f"{identity_map.get('risk_level')}\n"
    "This represents the total $USD value of the account, including positions, margin, and available funds.\n"
    "# Available Balance\n"
    f"{available_balance}\n"
    "Portions of this 'available_balance' can be used for placing new orders or modifying existing positions.\n"
    "Always leave a fraction of the total 'available_balance' as a safety buffer for unforeseen volatility.\n"
    "The 'available_balance' is shared by all positions, so it is important to keep track of the available value and adjust your position sizes accordingly.\n"
    "# Open Positions\n"
    f"{comp_dict}\n"
    "# Here is the most recent information I want to base my decisions on:\n"
    f"{formatted_data_summary}\n"
    "# Please provide a JSON dictionary matching the following Pydantic model for rebalance strategy, determining the optimal target allocations based on the market data provided:\n"
    """
    {
        "positions_to_maintain": [
            {
                "asset": "ETH",
                "size": 1500,
                "reasoning": "ETH has a high Sharpe Ratio and strong performance, aligning with the strategy to maximize returns."
            }
        ],
        "positions_to_modify": [
            {
                "asset": "BTC",
                "size": 0,
                "reasoning": "BTC's recent metrics suggest underperformance relative to market conditions."
            }
        ],
        "positions_to_open": [
            {
                "asset": "ADA",
                "size": 500,
                "reasoning": "ADA presents a promising growth opportunity aligned with current market trends."
            }
        ]
    }
    """
    "Provide only the JSON response without any additional text."
)


In [54]:
user_message

'# Instructions:\nHere are some details about my trading portfolio. Please help me make decisions to rebalance it based on the provided data.\n# Personality\nNone\n# Risk Level\nModerate risk, willing to risk some money for the right investments but not chasing every new opportunity.\nThis represents the total $USD value of the account, including positions, margin, and available funds.\n# Available Balance\n1.6195672674001798\nPortions of this \'available_balance\' can be used for placing new orders or modifying existing positions.\nAlways leave a fraction of the total \'available_balance\' as a safety buffer for unforeseen volatility.\nThe \'available_balance\' is shared by all positions, so it is important to keep track of the available value and adjust your position sizes accordingly.\n# Open Positions\n{\'LAVA\': 1.0, \'MAGIC\': 0.0, \'STG\': 0.0, \'USDC\': 0.0, \'DAI\': 0.0, \'ARB\': 0.0, \'USDT\': 0.0, \'CRV\': 0.0, \'ZRO\': 0.0}\n# Here is the most recent information I want to b

In [55]:
system_prompt = (
    "# Instructions:\n"
    "Act as a knowledgeable cryptocurrency assistant helping users manage and optimize their trading portfolio.\n"
    "Users understand that trading cryptocurrency is inherently risky and seek to make informed, strategic decisions to maximize their returns.\n"
    "You will be provided with the following data:\n"
    "- `available_balance`: Represents the user's total available USD value for making new trades.\n"
    "- `risk_level`: Indicates the user's risk tolerance (e.g., Low, Moderate, High).\n"
    "- `current_positions`: A dictionary mapping cryptocurrency symbols to their current USD value holdings (e.g., {'ETH': 1500, 'BTC': 1000}).\n"
    "- `market_data`: Recent performance metrics for each asset, including Sharpe Ratio, Excess Return, Latest Price, and 60-Day Return (e.g., {'ETH': {'sharpe_ratio': 1.5, 'excess_return': 0.05, 'latest_price': 3000, '60d_return': 20.0}, ...}).\n"
    
    "Your primary objective is to provide a rebalance strategy that aims to maximize the portfolio's returns while adhering to the following guidelines:\n"
    
    "## **Risk Management:**\n"
    "- Ensure that no individual trade exceeds the `available_balance`.\n"
    "- Maintain a safety buffer within the `available_balance` to handle market volatility and unforeseen events.\n"
    "- Respect the user's `risk_level` by adjusting trade sizes accordingly.\n"
    "- Avoid suggesting trades below 10 USD to ensure meaningful portfolio adjustments.\n"
    
    "## **Performance Optimization:**\n"
    "- Prioritize assets with higher Sharpe Ratios and Excess Returns to maximize risk-adjusted returns.\n"
    "- Consider reallocating funds from underperforming assets to those with strong recent performance metrics.\n"
    "- Ensure diversification to spread risk while targeting high-return opportunities.\n"
    
    "## **Trade Recommendations:**\n"
    "You may suggest the following actions:\n"
    "- **Create a New Position:** Track in the list ```positions_to_open```.\n"
    "- **Modify or Close an Existing Position:** Track in the list ```positions_to_modify```.\n"
    "- **Maintain an Existing Position:** Track in the list ```positions_to_maintain```.\n"
    
    "## **Fields for Each Recommendation:**\n"
    "- `asset`: The cryptocurrency symbol (e.g., ETH, BTC).\n"
    "- `size`: The size of the trade denominated in USD. It must be greater than 10 USD and should not use up the entire `available_balance`, leaving enough funds for risk management and flexibility.\n"
    "  \t- Example: 1500 # If the `available_balance` is 1500, use at most 1400 for trades, keeping 100 as a buffer.\n"
    "- `reasoning`: A concise explanation for the decision.\n"
    "  \t- Example: ['ETH has a high Sharpe Ratio and significant 60-day return, indicating strong performance.', 'BTC's recent metrics suggest underperformance relative to target allocations.', 'ADA presents a promising growth opportunity aligned with target allocations.']\n"
    
    "## **Example Recommendation:**\n"
    "- **Asset:** ETH\n"
    "- **Size:** 1500\n"
    "- **Reasoning:** 'ETH has a high Sharpe Ratio and significant 60-day return, indicating strong performance. Increasing position aligns with the strategy to maximize returns.'\n"
    
    "## **Output Format:**\n"
    "Provide a JSON dictionary with the following structure:\n"
    "```json\n"
    "{\n"
    "  \"positions_to_open\": [\n"
    "    {\n"
    "      \"asset\": \"ADA\",\n"
    "      \"size\": 500,\n"
    "      \"reasoning\": \"ADA presents a promising growth opportunity aligned with target allocations.\"\n"
    "    }\n"
    "  ],\n"
    "  \"positions_to_modify\": [\n"
    "    {\n"
    "      \"asset\": \"BTC\",\n"
    "      \"size\": 0,\n"
    "      \"reasoning\": \"BTC's recent metrics suggest underperformance relative to target allocations.\"\n"
    "    }\n"
    "  ],\n"
    "  \"positions_to_maintain\": [\n"
    "    {\n"
    "      \"asset\": \"ETH\",\n"
    "      \"size\": 1500,\n"
    "      \"reasoning\": \"ETH has a high Sharpe Ratio and strong performance, aligning with the strategy to maximize returns.\"\n"
    "    }\n"
    "  ]\n"
    "}\n"
    "```\n"
    
    "Provide only the JSON response without any additional text."
)


In [56]:
# append our messages to the chat
model = "gpt-4o-2024-08-06"
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_message},
]


In [57]:
import json


In [58]:
comp_dict

{'LAVA': 1.0,
 'MAGIC': 0.0,
 'STG': 0.0,
 'USDC': 0.0,
 'DAI': 0.0,
 'ARB': 0.0,
 'USDT': 0.0,
 'CRV': 0.0,
 'ZRO': 0.0}

In [59]:
from pydantic import BaseModel, Field
from typing import List


In [60]:
class Position(BaseModel):
    asset: str = Field(..., description="The cryptocurrency symbol, e.g., ETH, BTC")
    size: float = Field(..., description="The size of the trade in USD")
    reasoning: str = Field(..., description="The reasoning behind the trade decision")

# Define the PositionReasoning model
class PositionReasoning(BaseModel):
    positions_to_open: List[Position] = Field(
        default_factory=list,
        description="Positions to open."
    )
    positions_to_modify: List[Position] = Field(
        default_factory=list,
        description="Positions to modify or close."
    )
    positions_to_maintain: List[Position] = Field(
        default_factory=list,
        description="Positions to maintain without changes."
    )

In [61]:
completion = openai_client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    response_format=PositionReasoning,
)

try:
    # Access the content of the first choice
    response_content = completion.choices[0].message.content

    # Parse the JSON string into a Python dictionary
    result = json.loads(response_content)

    # Access the rebalance dictionary
    rebalance_dict = result.get("rebalance", {})

    print("Rebalance Strategy:", rebalance_dict)
except json.JSONDecodeError as e:
    print("Error decoding JSON:", e)
    print("Raw response:", completion.choices[0].message.content)

Rebalance Strategy: {}


In [62]:
completion.choices

[ParsedChoice[PositionReasoning](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[PositionReasoning](content='{"positions_to_open":[],"positions_to_modify":[],"positions_to_maintain":[{"asset":"LAVA","size":1.0,"reasoning":"Maintaining the minimal position size in LAVA due to insufficient available balance and moderate risk approach."}]}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[], parsed=PositionReasoning(positions_to_open=[], positions_to_modify=[], positions_to_maintain=[Position(asset='LAVA', size=1.0, reasoning='Maintaining the minimal position size in LAVA due to insufficient available balance and moderate risk approach.')])))]